# pip install pyotp qrcode[pil] pillow

In [47]:
import time
import os
from pathlib import Path
import pyotp
import qrcode
from PIL import Image

In [49]:
ACCOUNT_NAME = "student@example.com"   # label shown in app
ISSUER_NAME = "MyCourseAuth"           # issuer shown in app
OUT_DIR = Path("./")                   # where to save files
QR_FILENAME = OUT_DIR / "totp_qr.png"
EXAMPLE_SUBMIT = OUT_DIR / "totp_example.py"

In [51]:
def generate_secret():
    """Generate a base32 secret for TOTP."""
    return pyotp.random_base32()  # 16-char base32 by default

In [53]:
def create_totp(secret):
    """Return a TOTP object (30s step default)."""
    return pyotp.TOTP(secret)

In [55]:
def save_qr(provisioning_uri, path):
    """Generate and save a QR code PNG from an otpauth URI."""
    qr = qrcode.QRCode(border=2)
    qr.add_data(provisioning_uri)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    img.save(path)
    return path

In [57]:
def write_example_submission(secret, path):
    """Write a minimal example file students can attach as code submission."""
    content = f"""# totp_example.py
# Minimal example: generate and validate a TOTP using a given secret.
import pyotp
secret = '{secret}'
totp = pyotp.TOTP(secret)
print('Current OTP (server-side check):', totp.now())
# For validation example, prompt the user:
code = input('Enter OTP from your authenticator app: ').strip()
print('Valid' if totp.verify(code) else 'Invalid or expired')
"""
    path.write_text(content)
    return path

In [59]:
def main():
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    # 1) Generate secret and TOTP
    secret = generate_secret()
    print("=== TOTP demo ===")
    print("Base32 secret (keep this private):", secret)
    totp = create_totp(secret)
    print("TOTP provisioning (otpauth) URI example:")
    uri = totp.provisioning_uri(name=ACCOUNT_NAME, issuer_name=ISSUER_NAME)
    print(uri)
    print()

    # 2) Current OTP and verification demonstration
    current_code = totp.now()
    print("Current TOTP (30s window) (for demo only):", current_code)
    # Verify current (should be True)
    print("Verify current code =>", totp.verify(current_code))
    # Show verification with tolerance (valid_window)
    past_code = totp.at(int(time.time()) - 30)
    future_code = totp.at(int(time.time()) + 30)
    print("Example past code (t-30s):", past_code, "Verify with window=1 =>", totp.verify(past_code, valid_window=1))
    print("Example future code (t+30s):", future_code, "Verify with window=1 =>", totp.verify(future_code, valid_window=1))
    print()

    # 3) Save QR code for scanning by an authenticator app
    qr_path = save_qr(uri, QR_FILENAME)
    print(f"Saved QR code image to: {qr_path.resolve()}")
    print("Open that file and scan it with Google Authenticator / Authy on your phone.")
    print("If you prefer not to scan, you can enter the secret manually into the app.")
    print()

    # 4) Write example submission code
    ex_path = write_example_submission(secret, EXAMPLE_SUBMIT)
    print("Wrote example student code to:", ex_path.resolve())
    print()

    # 5) Interactive validation prompt (to produce a screenshot)
    print("Now you can scan the QR on your phone and fetch a TOTP code from the app.")
    user_code = input("Enter the code you see in your authenticator app: ").strip()
    if totp.verify(user_code):
        print("SUCCESS: The code is valid.")
    else:
        # Try with one-step window tolerance
        if totp.verify(user_code, valid_window=1):
            print("SUCCESS (valid within ±1 step).")
        else:
            print("FAIL: Invalid or expired code. Make sure your phone time is synchronized and you scanned the correct QR / secret.")
    print("\nDone. Files to attach for submission:")
    print(" - QR image:", qr_path.resolve())
    print(" - Example code:", ex_path.resolve())
    print("Suggested screenshots to capture:")
    print("  1) Terminal showing the printed secret, the provisioning URI and the verification result.")
    print("  2) Your authenticator app showing the account label and the 6-digit code.")
    print("  3) (Optional) The QR image file or the QR scanning confirmation on your phone.")

if __name__ == "__main__":
    main()

=== TOTP demo ===
Base32 secret (keep this private): 3PVIRNXC7O3NWTHEBS6ZFJMPFAYWUIDS
TOTP provisioning (otpauth) URI example:
otpauth://totp/MyCourseAuth:student%40example.com?secret=3PVIRNXC7O3NWTHEBS6ZFJMPFAYWUIDS&issuer=MyCourseAuth

Current TOTP (30s window) (for demo only): 765011
Verify current code => True
Example past code (t-30s): 653469 Verify with window=1 => True
Example future code (t+30s): 604005 Verify with window=1 => True

Saved QR code image to: C:\Users\Anahit.Gulyan\Documents\Anahit\YSU\BlockChain\Grigoryan\totp_qr.png
Open that file and scan it with Google Authenticator / Authy on your phone.
If you prefer not to scan, you can enter the secret manually into the app.

Wrote example student code to: C:\Users\Anahit.Gulyan\Documents\Anahit\YSU\BlockChain\Grigoryan\totp_example.py

Now you can scan the QR on your phone and fetch a TOTP code from the app.


Enter the code you see in your authenticator app:  780267


FAIL: Invalid or expired code. Make sure your phone time is synchronized and you scanned the correct QR / secret.

Done. Files to attach for submission:
 - QR image: C:\Users\Anahit.Gulyan\Documents\Anahit\YSU\BlockChain\Grigoryan\totp_qr.png
 - Example code: C:\Users\Anahit.Gulyan\Documents\Anahit\YSU\BlockChain\Grigoryan\totp_example.py
Suggested screenshots to capture:
  1) Terminal showing the printed secret, the provisioning URI and the verification result.
  2) Your authenticator app showing the account label and the 6-digit code.
  3) (Optional) The QR image file or the QR scanning confirmation on your phone.


IndentationError: unexpected indent (3680036903.py, line 2)